# PyTorch - homework 2: neural networks

-- Prof. Dorien Herremans

Please run the whole notebook with your code and submit the `.ipynb` file on eDimension that includes your answers [so after you run it]. 

In [2]:
from termcolor import colored

student_number="1004455"
student_name="Victoria Yong"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Victoria Yong, number: 1004455


 ## Question 1 -- XOR neural network [3pts]

a) Train an (at least) 2-layer neural network that can solve the XOR problem. Hint: be sure to check both this week and last week's lab. 

b) Check the predictions resulting from your model in the second code box below.


In [16]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [56]:
# load your data
features = torch.Tensor([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])
labels = torch.Tensor([[0, 1, 1, 0]]).view(-1, 1)

in_dim = features.size(1)

# name your model xor
def xor(epochs, feats, labels, in_dim, out_dim=1):
# define your model loss function, optimizer, etc. 
    model = nn.Sequential(
        nn.Linear(in_dim, 128),
        nn.Sigmoid(),
        nn.Linear(128, out_dim),
        nn.Sigmoid()
    ).cuda()

    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # initialize weights
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 1)
    
    # train the model
    for epoch in range(epochs):
        for j in range(feats.size(0)):
            idx = np.random.randint(feats.size(0))
            x = torch.autograd.Variable(feats[idx], requires_grad=False).cuda()
            y = torch.autograd.Variable(labels[idx], requires_grad=False).cuda()

            optimizer.zero_grad()

            pred = model(x)
            loss = loss_func(pred, y)
            loss.backward()
            optimizer.step()

        if epoch % 1000 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.cpu().data.numpy()}, ")
    
xor(5000, features, labels, in_dim)

Epoch: 0, Loss: 5.206602096557617, 
Epoch: 1000, Loss: 0.12951618432998657, 
Epoch: 2000, Loss: 0.015476944856345654, 
Epoch: 3000, Loss: 0.0019497793400660157, 
Epoch: 4000, Loss: 0.00011420901864767075, 


In [57]:
# test your model using the following functions (make sure the output is printed and saved when you submit this notebook):
# depending on how you defined your network you may need to slightly tweek the below prediction function

test = [[0,0],[0,1],[1,1],[1,0]]

for trial in test: 
  Xtest = torch.Tensor(trial)
  y_hat = xor(Xtest)

  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  print("{0} xor {1} = {2}".format(int(Xtest[0]), int(Xtest[1]), prediction))

TypeError: xor() missing 3 required positional arguments: 'feats', 'labels', and 'in_dim'

## Question 2  [2pts]

Imagine a neural network model for a multilabel classification task. 

a) Which loss function should you use?

b) The resulting trained modal has a high variance error. Give 4 possible solutions to improve the model. 


```
[your answer here, no coding required]

* answer A
* answer B
  - 1
  - 2
  - ...

```


## Question 3 - Improve hit classification [5pts]

Remember the hit predicton dataset from last week? 

a) Improve the model using a multiplayer perceptron. 

b) Make sure to run your models on the GPU. 

c) Tweek the hyperparameters such as number of nodes or layers, or other. Show two possible configurations and explain which works better and very briefly explain why this may be the case. 




In [ ]:
# code your model 1

In [ ]:
# evaluate model 1 (called model1 here)


TP import pandas as pd 

def run_evaluation(my_model):

  test = pd.read_csv('/content/herremans_hit_1030test.csv')
  labels = test.iloc[:,-1]
  test = test.drop('Topclass1030', axis=1)
  testdata = torch.Tensor(test.values)
  testlabels = torch.Tensor(labels.values).view(-1,1)

  TP = 0
  TN = 0
  FN = 0
  FP = 0

  for i in range(0, testdata.size()[0]): 
    # print(testdata[i].size())
    Xtest = torch.Tensor(testdata[i])
    y_hat = my_model(Xtest)
    
    if y_hat > 0.5:
      prediction = 1
    else: 
      prediction = 0

    if (prediction == testlabels[i]):
      if (prediction == 1):
        TP += 1
      else: 
        TN += 1

    else:
      if (prediction == 1):
        FP += 1
      else: 
        FN += 1

  print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
  print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
  rate = TP/(FN+TP)
  print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

run_evaluation(model1)

In [ ]:
# code your model 2

In [ ]:
# evaluate model 2 (called model2 here)

run_evaluation(model2)

Which works better and why do you think this may be (very briefly)? 


**[your answer here, also please summarise the differences between your two models]**

Additionally, submit your results [here](https://forms.gle/NtJJEE7Wm5ZRM3Je7) for 'Class specific accuracy of correctly predicting a hit song' and see if you got the best performance of the class! Good luck!